### Задание 2
Содержит переработанный код с учетом повторений, обусловленных дополнительными командами.
Код по обязательным пунктом - ниже (Задание 1)

In [71]:
documents = [
 {'type': 'passport', 'number': '2207 876234', 'name': 'Василий Гупкин'},
 {'type': 'invoice', 'number': '11-2', 'name': 'Геннадий Покемонов'},
 {'type': 'insurance', 'number': '10006', 'name': 'Аристарх Павлов'}
]

directories = {
 '1': ['2207 876234', '11-2'],
 '2': ['10006'],
 '3': []
}


def combine(docs, dirs):
    """
    Функция формирования словаря: Номер документа - прочие данные: Тип документа, Владелец, Номер полки.
    Объединение данных словаря directories и documents.
    """
    
    shelf_num_dict = {}
    num_dict = {}
    
    # формирование словаря: Номер документа - Номер полки
    for key, value in dirs.items():
        for num_el in value:
            if len(value) != 0:
                shelf_num_dict.setdefault(str(num_el), key) 
    
    # Дополнение словарей по документам данными о номере полки
    for doc in docs:
        doc['shelf'] = shelf_num_dict[str(doc['number'])]
    
    num_dict = {str(doc['number']): {'type': doc['type'], 'name': doc['name'], 'shelf': doc['shelf']} for doc in docs}
 
    return num_dict

def check_shelf_presence(shelf_n, dirs):
    """
    Проверка наличия полки для хранения
    """
    return dirs.get(shelf_n, 'Полка отсутствует')

def check_num_presence(doc_n, docs, dirs):
    """
    Проверка наличия документа в данных
    """
    
    return combine(docs, dirs).get(doc_n, 'Документ не найден в базе')

def current_shelf_list(dirs):
    """
    Возвращает текущий перечень полок
    """
    shelf_list_int = []
    
    for key in dirs.keys():
        shelf_list_int.append(int(key))
    shelf_list_int.sort()
    
    int_str_tranformation = lambda list_int: str(list_int)
    shelf_list_int = list(map(int_str_tranformation, shelf_list_int))
    shelf_str = 'Текущий перечень полок: ' + ', '.join(shelf_list_int)
    
    return shelf_str
    
def current_doc_list(result_string, docs, dirs):
    """
    Возвращает текущий перечень документов
    """
    print(result_string, 'Текущий список документов:', sep = '\n')
    command_l(docs, dirs)

def delete_doc_from_shelf(num_d, docs, dirs):
    """
    Удаление документа с полки
    """
    for key, value in dirs.items():
        if num_d in value:
            value.remove(num_d)
            break
    return dirs
    
def command_p_s(command_input, doc_num):
    """
    Функции вывода данных о владельце и номере полки документа по его номеру.
    
    Команды p и s объединены в одну функцию, поскольку имеют одинаковые входные данные - номер документа, 
    и похожий результат
    """
    
    num_dict = combine(documents, directories)
    result = check_num_presence(doc_num, documents, directories)
    
    if result == 'Документ не найден в базе':
        return print(result)
    else:
        if command_input == 'p':
            result = num_dict[str(doc_num)]['name']
            result_string = 'Владелец документа:'
        elif command_input == 's':
            result = num_dict[str(doc_num)]['shelf']
            result_string = 'Документ хранится на полке:'
        return print(result_string, result)

def command_l(docs, dirs):
    """
    Функция вывода всех данных по всем документам
    """
    
    for key, value in combine(docs, dirs).items():
        type_doc = value['type']
        name_doc = value['name']
        shelf_doc = value['shelf']
        print(f'№: {key}, тип: {type_doc}, владелец: {name_doc}, полка хранения: {shelf_doc}')

def command_ads_ds(command_input, shelf_num, dirs):
    """
    Функции добавления и удаления полок в зависимости от их наличия и заполненности.
    
    Команды ads и ds объединены в одну функцию, поскольку имеют одинаковые входные данные - номер полки, 
    и похожий результат
    """
    shelf_list_int = []
    
    if check_shelf_presence(shelf_num, dirs) == 'Полка отсутствует':
        if command_input == 'ads':
            dirs[shelf_num] = []
            result_string = 'Полка добавлена'
        elif command_input == 'ds':
            result_string = 'Такой полки не существует'
    else:
        if command_input == 'ads':
            result_string = 'Такая полка уже существует'
        elif command_input == 'ds':
            if len(dirs[str(shelf_num)]) != 0:
                result_string = 'На полке есть документы, удалите их перед удалением полки'
            else:
                del(dirs[str(shelf_num)])
                result_string = 'Полка удалена' 
           
    return print(f'{result_string}. {current_shelf_list(dirs)}.')
    
# Дополнительные функции
def command_ad(num_d, type_d, name_d, shelf_d, dirs, docs):
    """
    Функция добавления документа на существующую полку
    """
    if check_shelf_presence(shelf_d, dirs) == 'Полка отсутствует':
        result_string = 'Такой полки не существует. Добавьте полку командой ads.'
    else:
        docs.append({'type': type_d, 'number': num_d, 'name': name_d})
        dirs[shelf_d].append(num_d)
        result_string = 'Документ добавлен.'
    
    return current_doc_list(result_string, docs, dirs)

def command_d(num_d, dirs, docs):
    """
    Функция удаления документа из данных
    """
    result = check_num_presence(num_d, docs, dirs)
    
    if result == 'Документ не найден в базе':
        result_string = 'Документ не найден в базе'
    else:
        for doc in docs:
            if doc['number'] == num_d:
                docs.remove(doc)
                break
        result_string = 'Документ удален.'
    
    return current_doc_list(result_string, docs, delete_doc_from_shelf(num_d, docs, dirs))

def command_m(num_d, shelf_d, dirs, docs):
    """
    Функция смены полки хранения документа
    """
    result_num_doc = check_num_presence(num_d, docs, dirs)
    result_shelf_doc = check_shelf_presence(shelf_d, dirs)
    
    if result_num_doc == 'Документ не найден в базе':
        result_string = 'Документ не найден в базе'
        return current_doc_list(result_string, docs, dirs)
        
    else:
        if result_shelf_doc == 'Полка отсутствует':
            result_string = 'Такой полки не существует'
            print(f'{result_string}. {current_shelf_list(dirs)}.')
        else:
            delete_doc_from_shelf(num_d, docs, dirs)[shelf_d].append(num_d)
            result_string = 'Документ перемещен.'
            
            return current_doc_list(result_string, docs, dirs)
        
def command_enter(command_input):
    """
    Определяет, какую функцию запускать в зависимости от команды,
    Запрашивает доп. входные данные, если необходимо
    """
    
    if command_input == 'p' or command_input == 's':
        doc_num = str(input('Введите номер документа: '))
        while doc_num == '':
            doc_num = str(input('Введите номер документа: '))
        command_p_s(command_input, doc_num)
    
    elif command_input == 'l':
        command_l(documents, directories)
    
    elif command_input == 'ads' or command_input == 'ds':
        doc_shelf = str(input('Введите номер полки: '))
        while doc_shelf == '':
            doc_shelf = str(input('Введите номер полки: '))
        command_ads_ds(command_input, doc_shelf, directories)

# Доп. функции   
    elif command_input == 'ad':
        doc_num = str(input('Введите номер документа: '))
        while doc_num == '':
            doc_num = str(input('Введите номер документа: '))
        doc_type = str(input('Введите тип документа: '))
        while doc_type == '':
            doc_type = str(input('Введите тип документа: '))
        doc_name = str(input('Введите владельца документа: '))
        while doc_name == '':
            doc_name = str(input('Введите владельца документа: '))
        doc_shelf = str(input('Введите полку для хранения: '))
        while doc_shelf == '':
            doc_shelf = str(input('Введите полку для хранения: '))
        command_ad(doc_num, doc_type, doc_name, doc_shelf, directories, documents)
    
    elif command_input == 'd':
        doc_num = str(input('Введите номер документа: '))
        while doc_num == '':
            doc_num = str(input('Введите номер документа: '))
        command_d(doc_num, directories, documents)
    
    elif command_input == 'm':
        doc_num = str(input('Введите номер документа: '))
        while doc_num == '':
            doc_num = str(input('Введите номер документа: '))
        doc_shelf = str(input('Введите номер полки: '))
        while doc_shelf == '':
            doc_shelf = str(input('Введите номер полки: '))
        command_m(doc_num, doc_shelf, directories, documents)
    
def main():
    """
    Принимает код команды (p - владелец по номеру документа, s - номер полки по номеру документа, 
                           l - вся информация по всем документам, ads - добавление полки, ds - удаление полки,
                           ad - добавление нового документа в данные, d - удаление документа, m - смена полки документа,
                           q - выход из режима редактирования);
    Запускает функций по командам
    """
    command = input("Введите команду: ").lower()
    while command != 'q':    
        command_enter(command)
        command = input("Введите команду: ").lower()

# ОСНОВНАЯ КОМАНДА
main()

Введите команду: q


### Задание 1

In [44]:
documents = [
 {'type': 'passport', 'number': '2207 876234', 'name': 'Василий Гупкин'},
 {'type': 'invoice', 'number': '11-2', 'name': 'Геннадий Покемонов'},
 {'type': 'insurance', 'number': '10006', 'name': 'Аристарх Павлов'}
]

directories = {
 '1': ['2207 876234', '11-2'],
 '2': ['10006'],
 '3': []
}


# ФУНКЦИИ: 
# Перевод (целочисленного) типа данных в строковый
int_str_tranformation = lambda list_int: str(list_int)

def combine(docs, dirs):
    """
    Функция формирования словаря: Номер документа - прочие данные: Тип документа, Владелец, Номер полки.
    Объединение данных словаря directories и documents.
    """
    
    shelf_num_dict = {}
    num_dict = {}
    
    # формирование словаря: Номер документа - Номер полки
    for key, value in dirs.items():
        for num_el in value:
            if len(value) != 0:
                shelf_num_dict.setdefault(str(num_el), key) 
    
    # Дополнение словарей по документам данными о номере полки
    for doc in docs:
        doc['shelf'] = shelf_num_dict[str(doc['number'])]
    
    num_dict = {str(doc['number']): {'type': doc['type'], 'name': doc['name'], 'shelf': doc['shelf']} for doc in docs}

    return num_dict

def command_p_s(command_input, doc_num):
    """
    Функции вывода данных о владельце и номере полки документа по его номеру.
    
    Команды p и s объединены в одну функцию, поскольку имеют одинаковые входные данные - номер документа, 
    и похожий результат
    """
    
    combine(documents, directories)
    
    result = num_dict.get(str(doc_num), 'Документ не найден в базе')
    
    if result == 'Документ не найден в базе':
        return print(result)
    else:
        if command_input == 'p':
            result = num_dict.get(str(doc_num), 'Документ не найден в базе')['name']
            result_string = 'Владелец документа:'
        elif command_input == 's':
            result = num_dict.get(str(doc_num), 'Документ не найден в базе')['shelf']
            result_string = 'Документ хранится на полке:'
        return print(result_string, result)

def command_l():
    """
    Функция вывода всех данных по всем документам
    """
    
    combine(documents, directories)
    
    for key, value in num_dict.items():
        type_doc = value['type']
        name_doc = value['name']
        shelf_doc = value['shelf']
        print(f'№: {key}, тип: {type_doc}, владелец: {name_doc}, полка хранения: {shelf_doc}')

def command_ads_ds(command_input, shelf_num, dirs):
    """
    Функции добавления и удаления полок в зависимости от их наличия и заполненности.
    
    Команды ads и ds объединены в одну функцию, поскольку имеют одинаковые входные данные - номер полки, 
    и похожий результат
    """
    shelf_list_int = []
    shelf_presence = dirs.get(shelf_num, 'Полка отсутствует')

    if shelf_presence == 'Полка отсутствует':
        if command_input == 'ads':
            dirs[shelf_num] = []
            result_string = 'Полка добавлена'
        elif command_input == 'ds':
            result_string = 'Такой полки не существует'
    else:
        if command_input == 'ads':
            result_string = 'Такая полка уже существует'
        elif command_input == 'ds':
            if len(dirs[str(shelf_num)]) != 0:
                result_string = 'На полке есть документы, удалите их перед удалением полки'
            else:
                del(dirs[str(shelf_num)])
                result_string = 'Полка удалена' 
                
    for key in dirs.keys():
        shelf_list_int.append(int(key))
    
    shelf_list_int.sort()
    
    shelf_list_int = list(map(int_str_tranformation, shelf_list_int))

    shelf_str = ', '.join(shelf_list_int)
        
    return print(f'{result_string}. Текущий перечень полок: {shelf_str}')
          
def command_enter(command_input):
    """
    Определяет, какую функцию запускать в зависимости от команды,
    Запрашивает доп. входные данные, если необходимо
    """
    if command_input == 'p' or command_input == 's':
        doc_num = str(input('Введите номер документа: '))
        while doc_num == '':
            doc_num = str(input('Введите номер документа: '))
        command_p_s(command_input, doc_num)
    elif command_input == 'l':
        command_l()
    elif command_input == 'ads' or command_input == 'ds':
        shelf_num = str(input('Введите номер полки: '))
        while shelf_num == '':
            shelf_num = str(input('Введите номер полки: '))
        command_ads_ds(command_input, shelf_num, directories)

def main():
    """
    Ввод команды (p - владелец по номеру документа, s - номер полки по номеру документа, 
                  l - вся информация по всем документам, ads - добавление полки, ds - удаление полки);
    Запуск функций по командам
    """
    command = input("Введите команду: ").lower()
    while command != 'q':    
        command_enter(command)
        command = input("Введите команду: ").lower()

# ОСНОВНАЯ КОМАНДА
main()

Введите команду: q
